In diesem Notebook werden wird der Code gesammelt, der für die finale Abgabe der Masterarbeit nicht relevant ist, aber dennoch interessant sein könnte. 

In [ ]:
import pandas as pd
import json

#eigene Module
from src.dataimport import load_text


# JSON zu Pandas Dataframes
MajorClaims, Claims, Premises und Argumentationsbeziehungen werden in einzelne Dataframes umgewandelt. 

In [ ]:
def json_to_dataframe(json_data):
    data = json.loads(json_data)
    # Create a DataFrame for MajorClaims
    major_claims_df = pd.DataFrame(data["MajorClaims"])
    major_claims_df["Type"] = "MajorClaim"
    # Create a DataFrame for Claims
    claims_df = pd.DataFrame(data["Claims"])
    claims_df["Type"] = "Claim"
    # Create a DataFrame for Premises
    premises_df = pd.DataFrame(data["Premises"])
    premises_df["Type"] = "Premise"
    # Create a DataFrame for ArgumentativeRelations
    relations_df = pd.DataFrame(data["ArgumentativeRelations"])

    # for relations df calculate the types
    relations_df["OriginType"] = relations_df["Origin"].apply(lambda x: x[0])
    relations_df["TargetType"] = relations_df["Target"].apply(lambda x: x[0])

    return major_claims_df, claims_df, premises_df, relations_df

# Example usage
json_data = load_text(transformed_json_files[0])

major_claims_df, claims_df, premises_df, relations_df = json_to_dataframe(json_data)


```python
major_claims_df

ID	Text	Type
0	MC1	we should attach more importance to cooperatio...	MajorClaim
1	MC2	a more cooperative attitudes towards life is m...	MajorClaim


claims_df

ID	Text	Type
0	C1	through cooperation, children can learn about ...	Claim
1	C2	competition makes the society more effective	Claim
2	C3	without the cooperation, there would be no vic...	Claim


premises_df

ID	Text	Type
0	P1	What we acquired from team work is not only ho...	Premise
1	P2	During the process of cooperation, children ca...	Premise
2	P3	All of these skills help them to get on well w...	Premise
3	P4	the significance of competition is that how to...	Premise
4	P5	when we consider about the question that how t...	Premise
5	P6	Take Olympic games which is a form of competit...	Premise


relations_df

Origin	Relation	Target	OriginType
0	C1	For	MC	C
1	P1	supports	C1	P
2	P2	supports	C1	P
3	P3	supports	C1	P
4	C2	Against	MC	C
5	C3	For	MC	C
6	P6	supports	C3	P
7	P5	supports	C3	P
8	P4	supports	C2	P
```